In [ ]:
import numpy as np
import pandas as pd
import collections
import time

In [35]:
# Words that are answers
word_answers = list(np.loadtxt("wordle-answers-alphabetical.txt", comments="#", delimiter="\n", unpack=False, dtype=str))
# Words that are accepted, not including answers
word_allowed = list(np.loadtxt("wordle-allowed-guesses.txt", comments="#", delimiter="\n", unpack=False, dtype=str))

num_answers = len(word_answers)
num_allowed = len(word_allowed)

# Separate into letters
# E.g. letter[0] is the list of all first letters of the words
letter_answers = list(zip(*[list(_) for _ in word_answers]))
letter_allowed = list(zip(*[list(_) for _ in word_allowed]))

# Flattened list of letters (all letters regardless of position)
all_letter_answers = [item for sublist in letter_answers for item in sublist]
all_letter_allowed = [item for sublist in letter_allowed for item in sublist]

all_letters_pos = [list(letter_answers[_]) + list(letter_allowed[_]) for _ in range(5)]
all_letters = list(all_letter_answers) + list(all_letter_allowed)
all_words = word_answers + word_allowed

In [42]:
# print([all_letters_pos[_][11231] for _ in range(5)])
# print(all_words[11231])

['s', 'u', 'p', 'r', 'a']
supra


## Wordle game as a python function

In [ ]:
def wordle(target_word_idx: int, guess: str) -> dict:
    # For a chosen target word (identified by index), make a guess
    # This function will output in a dictionary:
    #   - "correct_guess" [boolean]: Whether the guess was correct or not,
    #   - "correct_letters" [set]: The letters in the guess that were correct
    #   - "incorrect_letters" [set]: The letters in the guess that were incorrect
    #   - "correct_letter_locations" [list]: The letters in the guess that were correct and in the right place,
    #       e.g. ['m', '', 'd', '', ''] for guessing "medal" for the target word "modem"

    if not (len(guess) == 5):
        raise AssertionError("Guess needs to have 5 letters")
    if not (guess in (word_answers + word_allowed)):
        raise AssertionError("Guess is not in the word list")

    target_word = word_answers[target_word_idx]
    target_letters = list(target_word)

    guess_letters = list(guess)

    correct_locations = [target_letters[j] ==
                         guess_letters[j] for j in range(5)]
    correct_letter_locations = [(guess_letters[j] if (target_letters[j] == guess_letters[j]) else "") for j in range(5)]

    correct_letters = set(target_letters).intersection(set(guess_letters))

    incorrect_letters = set(guess_letters) - correct_letters

    result = {"correct_guess": np.prod(correct_locations) == 1,
              "correct_letters": correct_letters,
              "incorrect_letters": incorrect_letters,
              "correct_letter_locations": correct_letter_locations}

    return result


target_word_idx = 1245
print(f"{word_answers[target_word_idx]}")
wordle(target_word_idx, "medal")


### Scoring systems

In [53]:

# letter_score [dict]: A scoring system for letters based on their popularity
count_letters = collections.Counter(all_letters)
letter_score = dict(count_letters.most_common())
# Divide through by the sum of all scores, so they all add to 1
total = sum(letter_score.values())
letter_score = {k: v / total for k, v in letter_score.items()}

# letter_score_pos [list(dict, dict, dict, dict, dict)]: A scoring system for letters based on their popularity in each position
letter_score_pos = []
for j in range(5):
    count_letters = collections.Counter(all_letters_pos[j])
    ls = dict(count_letters.most_common())
    # Divide through by the sum of all scores, so they all add to 1
    total = sum(ls.values())
    ls = {k: v / total for k, v in ls.items()}
    letter_score_pos.append(ls)

## Strategy 1
- Use all answers and allowed words
- Guess words based on banned letters and words only
- Uses letter score based on letter frequency regardless of position
    

In [54]:



# For each word, get the list of unique letters in it
word_letter_set = [list(set(list(_))) for _ in all_words]

def bestWord1(banned_letters: set = set(), banned_words: set = set(), correct_letters: set = set(), correct_letter_locations: list = ['', '', '', '', '']) -> str:
    # Make a copy of letter scoring without banned letters
    ls = dict(letter_score)
    for b in banned_letters:
        ls[b] = 0
    
    # Calculate the letter score for each word
    word_score = []
    for w in word_letter_set:
        score = sum([ls[_] for _ in w])
        word_score.append(score)

    # Get the word indices of the banned words
    banned_word_idx = [all_words.index(_) for _ in banned_words]
    # Set banned word scores to zero
    for j in banned_word_idx:
        word_score[j] = 0
    

    best_idx = np.argmax(word_score)

    best_word = all_words[best_idx]

    return best_word


# bestWord1(["a", "b", "c", "l"], ["osier"])
bestWord1()



'arose'

## Strategy 2
- Use all answers and allowed words
- Guess words based on banned letters and words only
- Uses letter score based on letter frequency in each position

In [55]:


def bestWord2(banned_letters: set = set(), banned_words: set = set(), correct_letters: set = set(), correct_letter_locations: list = ['', '', '', '', '']) -> str:
    # Make a copy of letter scoring without banned letters
    ls = list(letter_score_pos)
    for j in range(5):
        for b in banned_letters:
            ls[j][b] = 0
    


    # Calculate the letter score for each word
    word_score = []
    for w in all_words:
        w_letters = list(w)
        score = sum([ls[_][w_letters[_]] for _ in range(5)])
        word_score.append(score)

    # Get the word indices of the banned words
    banned_word_idx = [all_words.index(_) for _ in banned_words]
    # Set banned word scores to zero
    for j in banned_word_idx:
        word_score[j] = 0
    

    best_idx = np.argmax(word_score)

    best_word = all_words[best_idx]

    return best_word

bestWord2()

'sores'

In [ ]:
all_words

# Play the game
using strategy 1

In [52]:
target_word_idx = 1
print(f"Target word: {word_answers[target_word_idx]}\n")

banned_letters = set()
banned_words = set()
num_guesses = 0
guesses = []
while True:
    best_word = bestWord1(list(banned_letters), list(banned_words))
    guesses.append(best_word)
    num_guesses = num_guesses + 1

    result = wordle(target_word_idx, best_word)

    if result["correct_guess"]:
        break
    
    banned_letters = banned_letters.union(result["incorrect_letters"])
    banned_words = banned_words.union([best_word])
    # print(best_word)
    # print(banned_letters)
    # time.sleep(1)

print(f"Number of guesses: {num_guesses}")
print(guesses)


Target word: abase

Number of guesses: 1
['abase']


In [ ]:
# Function that takes the bestWord function as an input and plays every game of wordle (for all word_answers)
# def playWordle(bestWord, )